In [1]:
# conda activate spatialdata_Dec24_MM
import scanpy as sc
import matplotlib as mpl
import numpy as np
import pandas as pd
import anndata as ad
import squidpy as sq
import json
import matplotlib.pyplot as plt

import os
import scipy

import json
import time
from itertools import islice
import numpy as np
from shapely.geometry import Point

# polygons downloaded from https://wcm.app.box.com/s/z3uev1i43xbjc88rnrz31dta5xw9jxjv
# polygons at /Saha/Polygon_files


### Define functions

In [2]:
### Define functions 
import json
import time
from itertools import islice
import numpy as np
from shapely.geometry import Point


### Convert polygons to .json format
def convert_polygons_to_json(
    polygons_df,
    x_col='x_global_px',
    y_col='y_global_px',
    use_subset=True,
    subset_size=100,
    output_file=None,
    round_coords=False,
    centroid_scale_factor=1.0,
    polygon_scale_factor=1.0
):
    """
    Converts polygon data into a JSON format where each cell is an element containing
    a list of [x, y] coordinate pairs for all vertices. Optionally scales the centroids and polygons separately.

    Parameters:
        polygons_df (pd.DataFrame): The dataframe containing polygon data with indices as cell IDs.
        x_col (str): Column name for x-coordinates.
        y_col (str): Column name for y-coordinates.
        use_subset (bool): Whether to use a subset of the data for testing.
        subset_size (int): Number of rows to process if using a subset.
        output_file (str): Path to save the JSON file. If None, the JSON is not saved.
        round_coords (bool): Whether to round coordinates to 2 decimal points.
        centroid_scale_factor (float): Factor to scale centroids. Default is 1.0 (no scaling).
        polygon_scale_factor (float): Factor to scale polygons. Default is 1.0 (no scaling).

    Returns:
        dict: A dictionary representation of the JSON structure.
        float: Estimated time for processing the full dataset (if using a subset).
    """
    cell_polygons = {}

    if use_subset:
        data_to_process = polygons_df.iloc[:subset_size]
        print(f"Processing a subset of {subset_size} rows for testing...")
    else:
        data_to_process = polygons_df
        print("Processing the full dataset...")

    start_time = time.time()

    # Calculate the central point of all polygons
    central_x = data_to_process[x_col].mean()
    central_y = data_to_process[y_col].mean()
    central_point = Point(central_x, central_y)

    # Group the data by cell ID
    grouped_data = data_to_process.groupby(data_to_process.index)

    # Process and scale points for each cell
    for cell_id, group in grouped_data:
        vertices = []
        centroid_x = group[x_col].mean()
        centroid_y = group[y_col].mean()

        # Scale the centroid
        if centroid_scale_factor != 1.0:
            vector_x = centroid_x - central_point.x
            vector_y = centroid_y - central_point.y
            scaled_centroid_x = central_point.x + vector_x * centroid_scale_factor
            scaled_centroid_y = central_point.y + vector_y * centroid_scale_factor
        else:
            scaled_centroid_x, scaled_centroid_y = centroid_x, centroid_y

        for _, row in group.iterrows():
            x = row[x_col]
            y = row[y_col]

            # Scale the polygon vertices relative to the scaled centroid
            if polygon_scale_factor != 1.0:
                vector_x = x - centroid_x
                vector_y = y - centroid_y
                new_x = scaled_centroid_x + vector_x * polygon_scale_factor
                new_y = scaled_centroid_y + vector_y * polygon_scale_factor
            else:
                new_x, new_y = x, y

            if round_coords:
                new_x = round(new_x, 2)
                new_y = round(new_y, 2)

            vertices.append([new_x, new_y])

        cell_polygons[cell_id] = vertices

    end_time = time.time()
    elapsed_time = end_time - start_time

    estimated_time_full_dataset = None
    if use_subset:
        total_rows = len(polygons_df)
        estimated_time_full_dataset = (elapsed_time / subset_size) * total_rows
        print(f"\nTime taken for {subset_size} rows: {elapsed_time:.4f} seconds")
        print(f"Estimated time for full dataset ({total_rows} rows): {estimated_time_full_dataset:.2f} seconds")
    else:
        print(f"\nTime taken for full dataset: {elapsed_time:.4f} seconds")

    print("\nFirst few entries of cell_polygons:")
    for key, value in islice(cell_polygons.items(), 5):
        print(f"{key}: {value}")

    if output_file:
        with open(output_file, 'w') as f:
            json.dump(cell_polygons, f, indent=2)
        print(f"\nJSON saved to {output_file}")

    return cell_polygons, estimated_time_full_dataset

# Usage 
# full_json, _ = convert_polygons_to_json(
#     filtered_polygons,
#     x_col='CenterX_global_px',  # Replace with your desired x-coordinate column name
#     y_col='CenterY_global_px',  # Replace with your desired y-coordinate column name
#     use_subset=False,
#     output_file='obsSegmentations_polygons.json',
#     round_coords=True,
#     centroid_scale_factor=1,
#     polygon_scale_factor=1   
# )




### ADDING MOST VARIABLE GENES ###

import pandas as pd
import scipy.sparse

def extract_individual_components(adata, components, select_genes=None):
    """
    Extract specified data from an AnnData object and return a DataFrame.
    Also extracts a gene expression matrix for select_genes (user-supplied or top 30 variable genes).
    Stores the gene expression matrix in adata.uns['genes'].

    Parameters:
    adata (AnnData): The AnnData object containing the data.
    components (dict): A dictionary specifying the data to extract.
    select_genes (list, optional): List of gene names to extract as a gene expression matrix.

    Returns:
    pd.DataFrame: A DataFrame containing the extracted data.
    """
    extracted_data = {}

    for col_name, (location, key, *index) in components.items():
        if location == "obsm":
            extracted_data[col_name] = adata.obsm[key][:, index[0]] if index else adata.obsm[key]
        elif location == "obs":
            extracted_data[col_name] = adata.obs[key]
        elif location == "uns":
            extracted_data[col_name] = adata.uns[key]
        else:
            raise ValueError(f"Unsupported location '{location}'. Use 'obsm', 'obs', or 'uns'.")

    df = pd.DataFrame(extracted_data, index=adata.obs_names)

    # If select_genes is not provided, compute top 30 most variable genes
    if select_genes is None:
        sc.pp.highly_variable_genes(adata, n_top_genes=30)
        select_genes = adata.var[adata.var['highly_variable']].index.tolist()
        print("Top 30 variable genes:", select_genes)
    else:
        print("Using user-supplied select_genes:", select_genes)

    # Check if all selected genes are present in adata.var_names
    missing = [g for g in select_genes if g not in adata.var_names]
    if missing:
        print("Warning: These genes are missing from adata:", missing)
    else:
        print("All selected genes are present in adata.")

    # Extract expression matrix for selected genes
    if select_genes:  # Only proceed if the list is not empty
        try:
            if scipy.sparse.issparse(adata.X):
                exp_mtx_genes = pd.DataFrame(
                    adata[:, select_genes].X.toarray(),
                    columns=select_genes,
                    index=adata.obs_names
                )
            else:
                exp_mtx_genes = pd.DataFrame(
                    adata[:, select_genes].X,
                    columns=select_genes,
                    index=adata.obs_names
                )
            adata.uns['genes'] = exp_mtx_genes
            print(f"Shape of genes matrix: {exp_mtx_genes.shape}")
            print(f"First few rows of genes matrix:\n{exp_mtx_genes.head()}")
        except Exception as e:
            print(f"Error extracting gene expression matrix: {e}")
    else:
        print("No genes found to extract.")

    return df

def export_individual_components(df, export_config, output_path, adata=None):
    """
    Export selected columns from the DataFrame or gene expression matrix to separate CSV files.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing all the data.
    export_config (dict): A dictionary where keys are filenames and values are lists of column names to export,
                          or the string 'genes' to export the gene expression matrix.
    output_path (str): The directory where CSV files will be saved.
    adata (AnnData): The AnnData object containing the gene expression matrix in adata.uns['genes'].
    """
    # Ensure output directory exists
    os.makedirs(output_path, exist_ok=True)

    for filename, columns in export_config.items():
        file_path = os.path.join(output_path, filename)
        if isinstance(columns, list):
            missing_columns = [col for col in columns if col not in df.columns]
            if missing_columns:
                raise ValueError(f"Columns {missing_columns} are not found in the DataFrame.")
            selected_df = df[columns]
            selected_df.to_csv(file_path, index=False)
            print(f"Exported {columns} to {file_path}")
        elif columns == 'genes':
            if adata is not None and 'genes' in adata.uns:
                exp_mtx_genes = adata.uns['genes']
                # Try to include cell_id if present in df
                if 'cell_id' in df.columns:
                    exp_mtx_genes_with_id = pd.concat([df[['cell_id']], exp_mtx_genes], axis=1)
                else:
                    exp_mtx_genes_with_id = exp_mtx_genes
                exp_mtx_genes_with_id.to_csv(file_path, index=False)
                print(f"Exported gene expression matrix to {file_path} with shape {exp_mtx_genes_with_id.shape}")
            else:
                raise ValueError("Gene expression matrix (adata.uns['genes']) is not available.")
        else:
            raise ValueError(f"Invalid column specification for {filename}")




### Create .json Vitesse config 
def generate_json_file(version, name, description, dataset_uid, dataset_name, files, coordination_space, layout, init_strategy, output_file, url_prefix):
    # Update the URLs in the files list with the provided prefix
    for file in files:
        file['url'] = f"{url_prefix}/{file['url'].split('/')[-1]}"

    data = {
        "version": version,
        "name": name,
        "description": description,
        "datasets": [
            {
                "uid": dataset_uid,
                "name": dataset_name,
                "files": files
            }
        ],
        "coordinationSpace": coordination_space,
        "layout": layout,
        "initStrategy": init_strategy
    }

    with open(output_file, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"JSON file '{output_file}' generated successfully.")

# Example usage with the provided files
files = [
    {
        "fileType": "obsEmbedding.csv",
        "url": "obsEmbedding_pca.csv",
        "coordinationValues": {
            "obsType": "cell",
            "embeddingType": "PCA"
        },
        "options": {
            "obsIndex": "cell_id",
            "obsEmbedding": ["PC_1", "PC_2"]
        }
    },
    {
        "fileType": "obsEmbedding.csv",
        "url": "obsEmbedding_umap.csv",
        "coordinationValues": {
            "obsType": "cell",
            "embeddingType": "UMAP"
        },
        "options": {
            "obsIndex": "cell_id",
            "obsEmbedding": ["UMAP_1", "UMAP_2"]
        }
    },
    {
        "fileType": "obsSets.csv",
        "url": "obsSets_cell_anno.csv",
        "coordinationValues": {
            "obsType": "cell"
        },
        "options": {
            "obsIndex": "cell_id",
            "obsSets": [
                {
                    "name": "Cell Anno",
                    "column": "cell_type"
                }
            ]
        }
    },
    {
        "fileType": "obsFeatureMatrix.csv",
        "url": "exp_mtx_genes.csv",
        "coordinationValues": {
            "obsType": "cell",
            "featureType": "gene",
            "featureValueType": "expression"
        }
    },
    {
        "fileType": "obsSegmentations.json",
        "url": "obsSegmentations_polygons.json",
        "coordinationValues": {
            "obsType": "cell"
        }
    }
]

coordination_space = {
    "dataset": {"A": "D1"},
    "embeddingType": {"A": "UMAP", "B": "PCA"},
    "embeddingObsSetPolygonsVisible": {"A": False},
    "embeddingObsSetLabelsVisible": {"A": True},
    "embeddingObsSetLabelSize": {"A": 16},
    "embeddingObsRadiusMode": {"A": "manual"},
    "embeddingObsRadius": {"A": 3},
    "embeddingZoom": {"TSNE": 3, "UMAP": 3},
    "spatialZoom": {"A": None},
    "spatialTargetX": {"A": None},
    "spatialTargetY": {"A": None},
    "spatialSegmentationLayer": {
        "A": {
            "opacity": 1,
            "radius": 0,
            "visible": True,
            "stroked": False
        }
    },
    "obsColorEncoding": {"A": "cellSetSelection", "B": "geneSelection"}
}

layout = [
    {
        "component": "scatterplot",
        "coordinationScopes": {"dataset": "A", "embeddingType": "A"},
        "x": 0,
        "y": 0,
        "w": 4,
        "h": 8
    },
    {
        "component": "scatterplot",
        "coordinationScopes": {"dataset": "A", "embeddingType": "B"},
        "x": 0,
        "y": 4,
        "w": 4,
        "h": 8
    },
    {
        "component": "spatial",
        "props": {"cellRadius": 0.34},
        "coordinationScopes": {
            "spatialZoom": "A",
            "spatialTargetX": "A",
            "spatialTargetY": "A",
            "spatialSegmentationLayer": "A"
        },
        "x": 4,
        "y": 0,
        "w": 4,
        "h": 16
    },
    {
        "component": "obsSets",
        "coordinationScopes": {"dataset": "A"},
        "x": 0,
        "y": 8,
        "w": 4,
        "h": 8
    },
    {
        "component": "featureList",
        "x": 4,
        "y": 8,
        "w": 4,
        "h": 8
    },
    {
        "component": "heatmap",
        "props": {"transpose": True},
        "x": 8,
        "y": 8,
        "w": 4,
        "h": 18
    }
]

# ## Usage
# generate_json_file(
#     version="1.0.17",
#     name="Spatial Transcriptomics Visualization",
#     description="SAHA",
#     dataset_uid="D1",
#     dataset_name="Spatial Transcriptomics Dataset",
#     files=files,
#     coordination_space=coordination_space,
#     layout=layout,
#     init_strategy="auto",
#     url_prefix = f("https://mmaycon.github.io/Spatial_Data_Visualization/Datasets/SAHA/{slide}"),
#     output_file="VitConfig_fgithub.json"
# )



### Run it for multiple slides (PROTEIN)

In [ ]:
# Input file paths (adjust if needed)
adata_path = "/mnt/scratch2/Luke/SAHAMaxFuse/SAHA_All_PRT_share.h5ad" # protein data
polygon_dir = "/Saha/Polygon_files/"

# # Load AnnData once outside the loop
# if 'saha_adata' not in globals():
#     print("Loading AnnData...")
#     saha_adata = ad.read_h5ad(adata_path)
# else:
#     print("saha_adata already loaded, skipping read.")

saha_adata = ad.read_h5ad(adata_path)

# Your slides list
slides = saha_adata.obs['SAHA_name'].unique().tolist()[16]  # Add your slide names here
if not isinstance(slides, list):
    slides = [slides]

for slide in slides:
    print(f"\nProcessing slide: {slide}")

    # Slide-dependent paths
    polygon_file_pattern = f"{slide}-polygons.csv.gz"
    polygon_path = os.path.join(polygon_dir, polygon_file_pattern)
    output_dir = os.path.join(f"/Saha/Vitessce_view/Round_1/Objects/Datasets/SAHA/PROTEIN/{slide}_vit")

    os.makedirs(output_dir, exist_ok=True)

    # 2. Filter AnnData by slide
    filtered_adata = saha_adata[saha_adata.obs['SAHA_name'] == slide, :]

    # 3. Load and filter polygons
    if os.path.exists(polygon_path):
        polygons = pd.read_csv(polygon_path)
        cell = filtered_adata.obs['cell_id'].unique()
        filtered_polygons = polygons[polygons['cell'].isin(cell)]
        filtered_polygons = filtered_polygons.set_index('cell', drop=False)
    else:
        print(f"Polygon file not found: {polygon_path}")
        filtered_polygons = pd.DataFrame()
        # Write a failure .txt file in the output directory
        fail_txt_path = os.path.join(output_dir, f"failed_{slide}.txt")
        with open(fail_txt_path, "w") as fail_file:
            fail_file.write(f"Polygon file not found for slide: {slide}\nPath: {polygon_path}\n")
        # Skip to the next slide
        continue

    print(filtered_polygons)
           
    # 4. Subset AnnData to only cells with matching polygons
    if not filtered_polygons.empty:
        matching_cell_ids = filtered_polygons['cell'].unique()
        filtered_adata = filtered_adata[filtered_adata.obs['cell_id'].isin(matching_cell_ids), :]
    else:
        print("No polygons to filter.")

    # 5. Visualization (if polygon coordinates exist)
    if not filtered_polygons.empty and {'x_global_px', 'y_global_px'}.issubset(filtered_polygons.columns):
        plt.figure(figsize=(8, 8))
        plt.scatter(filtered_polygons['x_global_px'], filtered_polygons['y_global_px'], c='blue', alpha=0.5, label='Cells')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title(f'Filtered Cell Polygons: {slide}')
        plt.legend()
        plt.show()
    else:
        print("No polygon coordinates to plot.")

    # 6. Convert polygons to .json format
    output_json_path = os.path.join(output_dir, "obsSegmentations_polygons.json")
    full_json, _ = convert_polygons_to_json(
        filtered_polygons,
        x_col='x_global_px',
        y_col='y_global_px',
        use_subset=False,
        output_file=output_json_path,
        round_coords=False,
        centroid_scale_factor=1,
        polygon_scale_factor=1
    )

    # 7. Export other View types components .csv off adata 
    components = {
        'UMAP_1': ('obsm', 'X_umap_after_harmony', 0),
        'UMAP_2': ('obsm', 'X_umap_after_harmony', 1),
        'PC_1': ('obsm', 'X_pca', 0),
        'PC_2': ('obsm', 'X_pca', 1),
        # 'leiden_cluster': ('obs', 'leiden'),
        'cell_type': ('obs', 'celltype_mf'),
        'cell_id': ('obs', 'cell')
    }
    df = extract_individual_components(filtered_adata, components, select_genes=list(filtered_adata.var_names))
    print(df.head())

    export_config = {
        'obsEmbedding_umap.csv': ['cell_id', 'UMAP_1', 'UMAP_2'],
        'obsEmbedding_pca.csv': ['cell_id', 'PC_1', 'PC_2'],
        'obsSets_cell_anno.csv': ['cell_id', 'cell_type'],
        'exp_mtx_genes.csv': 'genes'
    }
    export_individual_components(df, export_config, output_path=output_dir, adata=filtered_adata)

    # 8. Generate the JSON file
    generate_json_file(
        version="1.0.17",
        name="Spatial Transcriptomics Visualization",
        description="SAHA",
        dataset_uid="D1",
        dataset_name="Spatial Transcriptomics Dataset",
        files=files,
        coordination_space=coordination_space,
        layout=layout,
        init_strategy="auto",
        url_prefix = f"https://mmaycon.github.io/Spatial_Data_Visualization/Datasets/SAHA/{slide}",
        output_file=f"{output_dir}/VitConfig_fgithub.json"
    )

### Run it for multiple slides (RNA)


In [ ]:
# # Input file paths (adjust if needed)
# adata_path = '/mnt/scratch2/Luke/SAHAMaxFuse/SAHA_All_RNA_share.h5ad' # RNA data
# polygon_dir = "/Saha/Polygon_files/"

# # # Load AnnData once outside the loop
# # if 'saha_adata' not in globals():
# #     print("Loading AnnData...")
# #     saha_adata = ad.read_h5ad(adata_path)
# # else:
# #     print("saha_adata already loaded, skipping read.")

# saha_adata = ad.read_h5ad(adata_path)

# # Your slides list
# slides = saha_adata.obs['SAHA_name'].unique().tolist()[:]  # Add your slide names here

# for slide in slides:
#     print(f"\nProcessing slide: {slide}")

#     # Slide-dependent paths
#     polygon_file_pattern = f"{slide}-polygons.csv.gz"
#     polygon_path = os.path.join(polygon_dir, polygon_file_pattern)
#     output_dir = os.path.join(f"/Saha/Vitessce_view/Round_1/Objects/Datasets/SAHA/RNA/{slide}_vit")

#     os.makedirs(output_dir, exist_ok=True)

#     # 2. Filter AnnData by slide
#     filtered_adata = saha_adata[saha_adata.obs['SAHA_name'] == slide, :]

#     # 3. Load and filter polygons
#     if os.path.exists(polygon_path):
#         polygons = pd.read_csv(polygon_path)
#         cell = filtered_adata.obs['cell_id'].unique()
#         filtered_polygons = polygons[polygons['cell'].isin(cell)]
#         filtered_polygons = filtered_polygons.set_index('cell', drop=False)
#     else:
#         print(f"Polygon file not found: {polygon_path}")
#         filtered_polygons = pd.DataFrame()
#         # Write a failure .txt file in the output directory
#         fail_txt_path = os.path.join(output_dir, f"failed_{slide}.txt")
#         with open(fail_txt_path, "w") as fail_file:
#             fail_file.write(f"Polygon file not found for slide: {slide}\nPath: {polygon_path}\n")
#         # Skip to the next slide
#         continue

#     print(filtered_polygons)
           
#     # 4. Subset AnnData to only cells with matching polygons
#     if not filtered_polygons.empty:
#         matching_cell_ids = filtered_polygons['cell'].unique()
#         filtered_adata = filtered_adata[filtered_adata.obs['cell_id'].isin(matching_cell_ids), :]
#     else:
#         print("No polygons to filter.")

#     # 5. Visualization (if polygon coordinates exist)
#     if not filtered_polygons.empty and {'x_global_px', 'y_global_px'}.issubset(filtered_polygons.columns):
#         plt.figure(figsize=(8, 8))
#         plt.scatter(filtered_polygons['x_global_px'], filtered_polygons['y_global_px'], c='blue', alpha=0.5, label='Cells')
#         plt.xlabel('X')
#         plt.ylabel('Y')
#         plt.title(f'Filtered Cell Polygons: {slide}')
#         plt.legend()
#         plt.show()
#     else:
#         print("No polygon coordinates to plot.")

#     # 6. Convert polygons to .json format
#     output_json_path = os.path.join(output_dir, "obsSegmentations_polygons.json")
#     full_json, _ = convert_polygons_to_json(
#         filtered_polygons,
#         x_col='x_global_px',
#         y_col='y_global_px',
#         use_subset=False,
#         output_file=output_json_path,
#         round_coords=False,
#         centroid_scale_factor=1,
#         polygon_scale_factor=1
#     )

#     # 7. Export other View types components .csv off adata 
#     components = {
#         'UMAP_1': ('obsm', 'X_harmony', 0),
#         'UMAP_2': ('obsm', 'X_harmony', 1),
#         'PC_1': ('obsm', 'X_pca', 0),
#         'PC_2': ('obsm', 'X_pca', 1),
#         # 'leiden_cluster': ('obs', 'leiden'),
#         'cell_type': ('obs', 'Insitutype_Broad'),
#         'cell_id': ('obs', 'cell')
#     }
#     df = extract_individual_components(filtered_adata, components, select_genes=list(filtered_adata.var_names))
#     print(df.head())

#     export_config = {
#         'obsEmbedding_umap.csv': ['cell_id', 'UMAP_1', 'UMAP_2'],
#         'obsEmbedding_pca.csv': ['cell_id', 'PC_1', 'PC_2'],
#         'obsSets_cell_anno.csv': ['cell_id', 'cell_type'],
#         'exp_mtx_genes.csv': 'genes'
#     }
#     export_individual_components(df, export_config, output_path=output_dir, adata=filtered_adata)

#     # 8. Generate the JSON file
#     generate_json_file(
#         version="1.0.17",
#         name="Spatial Transcriptomics Visualization",
#         description="SAHA",
#         dataset_uid="D1",
#         dataset_name="Spatial Transcriptomics Dataset",
#         files=files,
#         coordination_space=coordination_space,
#         layout=layout,
#         init_strategy="auto",
#         url_prefix = f"https://mmaycon.github.io/Spatial_Data_Visualization/Datasets/SAHA/{slide}",
#         output_file=f"{output_dir}/VitConfig_fgithub.json"
#     )

In [18]:
# from cmd
# > jupyter nbconvert --to script /Saha/Vitessce_view/Round_1/Scripts/Get_Vitessce_view_files.ipynb
# > conda activate spatialdata_Dec24_MM 
# > python /Saha/Vitessce_view/Round_1/Scripts/Get_Vitessce_view_files.py